<a href="https://colab.research.google.com/github/bhargav23/EVA4/blob/master/S4/EVA4_S4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #input 28x28x1, output 28x28x16, RF 3
        self.bn1 = nn.BatchNorm2d(16)
        self.do1 = nn.Dropout(0.15)

        self.conv2 = nn.Conv2d(16, 32, 3, padding=1) #input 28x28x16, output 26x28x32, RF 5
        self.bn2 = nn.BatchNorm2d(32)
        self.do2 = nn.Dropout(0.15)

        self.conv3 = nn.Conv2d(32, 16, 3) #input 28x28x32, output 26x26x16, RF 7
        self.bn3 = nn.BatchNorm2d(16)
        self.do3 = nn.Dropout(0.15)


        self.pool1 = nn.MaxPool2d(2, 2) # input 26x26*16 output 13*13*16 RF 8


        self.conv4 = nn.Conv2d(16, 16, 3) #input 13*13*16 output 11*11*16 RF 12
        self.bn4 = nn.BatchNorm2d(16)
        self.do4 = nn.Dropout(0.1)

        self.conv8 = nn.Conv2d(16, 16, 3) #input 9*9*16 output 7*7*16 RF 16
        self.bn8 = nn.BatchNorm2d(16)
        self.do8 = nn.Dropout(0.1)



        self.conv5 = nn.Conv2d(16, 16, 3) # input 5*5*16 op 3*3*16 RF 20
        self.bn5 = nn.BatchNorm2d(16)
        self.do5 = nn.Dropout(0.1)



        self.conv7 = nn.Conv2d(16, 10, 3) # input 3*3*16 op 3*3*10 RF 24

    def forward(self, x):

        x = F.relu(self.bn1(self.do1(self.conv1(x))))
        x = F.relu(self.bn2(self.do2(self.conv2(x))))
        x = self.pool1(F.relu(self.bn3(self.do3(self.conv3(x)))))
        x = F.relu(self.bn4(self.do4(self.conv4(x))))
        x = F.relu(self.bn8(self.do8(self.conv8(x))))
        x = F.relu(self.bn5(self.do5(self.conv5(x))))
        x = self.conv7(x)
        x = F.avg_pool2d(x, 3)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
           Dropout-2           [-1, 16, 28, 28]               0
       BatchNorm2d-3           [-1, 16, 28, 28]              32
            Conv2d-4           [-1, 32, 28, 28]           4,640
           Dropout-5           [-1, 32, 28, 28]               0
       BatchNorm2d-6           [-1, 32, 28, 28]              64
            Conv2d-7           [-1, 16, 26, 26]           4,624
           Dropout-8           [-1, 16, 26, 26]               0
       BatchNorm2d-9           [-1, 16, 26, 26]              32
        MaxPool2d-10           [-1, 16, 13, 13]               0
           Conv2d-11           [-1, 16, 11, 11]           2,320
          Dropout-12           [-1, 16, 11, 11]               0
      BatchNorm2d-13           [-1, 16, 11, 11]              32
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    print(epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0788, Accuracy: 9768/10000 (97.68%)

2


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0223, Accuracy: 9940/10000 (99.40%)

19


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0233, Accuracy: 9933/10000 (99.33%)

20


loss=0.0041741132736206055 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 41.33it/s]



Test set: Average loss: 0.0236, Accuracy: 9926/10000 (99.26%)

